In [1]:
from enum import Enum
from dataclasses import dataclass
from rich import print
from typing import TypeVar

In [3]:
# code de la séance précédente

class Rive(Enum):
    GAUCHE = "gauche"
    DROITE = "droite"
 
# frozen pour pouvoir utiliser un Etat comme clef de dictionnaire
@dataclass(frozen=True)
class Etat:
    berger: Rive
    loup: Rive
    mouton: Rive
    chou: Rive
    
DEPART = Etat(
    berger=Rive.GAUCHE, 
    loup=Rive.GAUCHE,
    mouton=Rive.GAUCHE,
    chou=Rive.GAUCHE,
)

ARRIVEE = Etat(
    berger=Rive.DROITE, 
    loup=Rive.DROITE,
    mouton=Rive.DROITE,
    chou=Rive.DROITE,
)

etats = list()
for cote_berger in (Rive.GAUCHE, Rive.DROITE):
    for cote_loup in (Rive.GAUCHE, Rive.DROITE):
        for cote_mouton in (Rive.GAUCHE, Rive.DROITE):
            for cote_chou in (Rive.GAUCHE, Rive.DROITE):
                etats.append(
                    Etat(
                        berger=cote_berger,
                        loup=cote_loup,
                        mouton=cote_mouton,
                        chou = cote_chou,
                    )
                )
                
def est_valide(etat: Etat) -> bool:
    if etat.loup == etat.mouton and etat.loup != etat.berger:
        return False
    if etat.mouton == etat.chou and etat.mouton != etat.berger:
        return False
    return True

assert est_valide(etat=DEPART)
probleme = Etat(
    berger=Rive.GAUCHE,
    loup=Rive.DROITE,
    mouton=Rive.DROITE,
    chou=Rive.GAUCHE,
)
assert not est_valide(etat=probleme)
SOMMETS = [etat for etat in etats if est_valide(etat)]
def sont_relies(etat1: Etat, etat2: Etat) -> bool:
    if etat1.berger == etat2.berger:
        return False
    nombre_changements = 0
    if etat1.loup != etat2.loup:
        nombre_changements = nombre_changements + 1
    if etat1.mouton != etat2.mouton:
        nombre_changements = nombre_changements + 1
    if etat1.chou != etat2.chou:
        nombre_changements = nombre_changements + 1
    if nombre_changements < 2:
        return True
    else:
        return False
    
assert not sont_relies(
    etat1=Etat(
        berger=Rive.GAUCHE,
        loup=Rive.GAUCHE,
        mouton=Rive.GAUCHE,
        chou=Rive.GAUCHE,
    ),
    etat2=Etat(
        berger=Rive.GAUCHE,
        loup=Rive.GAUCHE,
        mouton=Rive.GAUCHE,
        chou=Rive.GAUCHE,
    ),
)
assert sont_relies(
    etat1=Etat(
        berger=Rive.GAUCHE,
        loup=Rive.GAUCHE,
        mouton=Rive.GAUCHE,
        chou=Rive.GAUCHE,
    ),
    etat2=Etat(
        berger=Rive.DROITE,
        loup=Rive.GAUCHE,
        mouton=Rive.DROITE,
        chou=Rive.GAUCHE,
    ),
)

ARRETES = [
    (sommet1, sommet2)
    for sommet1 in SOMMETS
    for sommet2 in SOMMETS
    if sont_relies(sommet1, sommet2)
]
assert len(ARRETES) == 20

## Début séance 02

In [4]:
S = TypeVar("Sommet")

In [5]:
def recupere_voisins(sommet: S, arretes: list[tuple[S, S]]) -> list[S]:
    resultat = list()
    for sommet1, sommet2 in arretes:
        if sommet1 == sommet:
            resultat.append(sommet2)
    return resultat

In [6]:
assert recupere_voisins(sommet=1, arretes=[(1, 2), (3, 1)]) == [2]
assert (
    recupere_voisins(
        sommet=2, 
        arretes=[(1, 2), (2, 3), (2, 5), (3, 4), (4, 5), (5, 6), (6, 7), (8, 6)]) 
    == 
    [3, 5]
)

In [7]:
def sont_connectes(
    depart: S, 
    arrivee: S,
    sommets: list[S],
    arretes: list[tuple[S, S]],
    debug: bool=False,
) -> bool:
    a_visiter = [depart]
    visites = []
    while a_visiter:
        if debug:
            print(f"{a_visiter=}")
            print(f"{visites=}")
            print()
        sommet_courant = a_visiter.pop()
        visites.append(sommet_courant)
        for voisin in recupere_voisins(sommet=sommet_courant, arretes=arretes):
            if voisin not in visites:
                a_visiter.append(voisin)
        if sommet_courant == arrivee:
            return True
    return False

In [8]:
sommets = [1, 2, 3, 4, 5, 6, 7, 8]
arretes = [(1, 2), (2, 3), (2, 5), (3, 4), (4, 5), (5, 6), (6, 7), (8, 6)]
assert sont_connectes(depart=1, arrivee=7, sommets=sommets, arretes=arretes)
assert not sont_connectes(depart=1, arrivee=8, sommets=sommets, arretes=arretes)

In [9]:
sont_connectes(depart=1, arrivee=8, sommets=sommets, arretes=arretes, debug=True)

a_visiter=[1]

visites=[]

a_visiter=[2]

visites=[1]

a_visiter=[3, 5]

...

a_visiter=[3]

visites=[1, 2, 5, 6, 7]

a_visiter=[4]

visites=[1, 2, 5, 6, 7, 3]

False

## Exercice

Transformez la fonction `sont_connectes` en une fonction `cherche_chemin` qui renvoie un chemin reliant les deux états s'il en existe un.

**Indication** construire au fur et à mesure un dictionnaire représentant la relation entre sommets *a été vu en premier par*.

In [10]:
def determine_chemin(
    depart: S, 
    arrivee: S, 
    vu_en_premier_par: dict[S, S | None]
) -> list[S]:
    resultat_intermediaire = list()
    sommet_courant = arrivee
    while sommet_courant is not None:
        resultat_intermediaire.append(sommet_courant)
        sommet_courant = vu_en_premier_par[sommet_courant]
    return [sommet for sommet in reversed(resultat_intermediaire)]

In [11]:
assert determine_chemin(
    depart=1, 
    arrivee=7, 
    vu_en_premier_par={
        1: None,
        2: 1,
        3 : 2,
        5: 2,
        6: 5,
        7: 6
    }
) == [1, 2, 5, 6, 7]

In [12]:
class PasDeChemin(Exception):
    ...

In [13]:
def cherche_chemin(
    depart: S,
    arrivee: S,
    sommets: list[S],
    arretes: list[tuple[S, S]],
    debug: bool = False,
) -> list[S]:
    a_visiter = [depart]
    visites = []
    vu_en_premier_par = dict()
    vu_en_premier_par[depart] = None
    while a_visiter:
        if debug:
            print(f"{a_visiter=}")
            print(f"{visites=}")
            print()
        sommet_courant = a_visiter.pop()
        visites.append(sommet_courant)
        for voisin in recupere_voisins(sommet=sommet_courant, arretes=arretes):
            if voisin not in visites:
                a_visiter.append(voisin)
                vu_en_premier_par[voisin] = sommet_courant
        if sommet_courant == arrivee:
            return determine_chemin(
                depart=depart, 
                arrivee=arrivee, 
                vu_en_premier_par=vu_en_premier_par
        )
    raise PasDeChemin

In [14]:
sommets = [1, 2, 3, 4, 5, 6, 7, 8]
arretes = [(1, 2), (2, 3), (2, 5), (3, 4), (4, 5), (5, 6), (6, 7), (8, 6)]
assert (
    cherche_chemin(depart=1, arrivee=7, sommets=sommets, arretes=arretes) 
    == 
    [1, 2, 5, 6, 7]
)

In [15]:
sommets = [1, 2, 3, 4, 5, 6, 7, 8]
arretes = [(1, 2), (2, 3), (2, 5), (3, 4), (4, 5), (5, 6), (6, 7), (8, 6)]
assert (
    cherche_chemin(depart=1, arrivee=8, sommets=sommets, arretes=arretes) 
    == 
    [1, 2, 5, 6, 7]
)

PasDeChemin: 

## Résolution du problème de départ

In [17]:
resultat = cherche_chemin(
    depart=DEPART,
    arrivee=ARRIVEE,
    sommets=SOMMETS,
    arretes=ARRETES,
)

In [18]:
print(resultat)

[
    Etat(
        berger=<Rive.GAUCHE: 'gauche'>,
        loup=<Rive.GAUCHE: 'gauche'>,
        mouton=<Rive.GAUCHE: 'gauche'>,
        chou=<Rive.GAUCHE: 'gauche'>
    ),
    Etat(
        berger=<Rive.DROITE: 'droite'>,
        loup=<Rive.GAUCHE: 'gauche'>,
        mouton=<Rive.DROITE: 'droite'>,
        chou=<Rive.GAUCHE: 'gauche'>
    ),
    Etat(
        berger=<Rive.GAUCHE: 'gauche'>,
        loup=<Rive.GAUCHE: 'gauche'>,
        mouton=<Rive.DROITE: 'droite'>,
        chou=<Rive.GAUCHE: 'gauche'>
    ),
    Etat(
        berger=<Rive.DROITE: 'droite'>,
        loup=<Rive.DROITE: 'droite'>,
        mouton=<Rive.DROITE: 'droite'>,
        chou=<Rive.GAUCHE: 'gauche'>
    ),
    Etat(
        berger=<Rive.GAUCHE: 'gauche'>,
        loup=<Rive.DROITE: 'droite'>,
        mouton=<Rive.GAUCHE: 'gauche'>,
        chou=<Rive.GAUCHE: 'gauche'>
    ),
    Etat(
        berger=<Rive.DROITE: 'droite'>,
        loup=<Rive.DROITE: 'droite'>,
        mouton=<Rive.GAUCHE: 'gauche'>,
        chou=<Rive.DROITE: 'droite'>
    ),
    Etat(
        berger=<Rive.GAUCHE: 'gauche'>,
        loup=<Rive.DROITE: 'droite'>,
        mouton=<Rive.GAUCHE: 'gauche'>,
        chou=<Rive.DROITE: 'droite'>
    ),
    Etat(
        berger=<Rive.DROITE: 'droite'>,
        loup=<Rive.DROITE: 'droite'>,
        mouton=<Rive.DROITE: 'droite'>,
        chou=<Rive.DROITE: 'droite'>
    )
]

In [21]:
code = {etat: indice for indice, etat in enumerate(SOMMETS)}

In [22]:
traduction = [(code[e1], code[e2]) for e1, e2 in ARRETES]

In [23]:
print(traduction)

[
    (0, 5),
    (1, 6),
    (1, 7),
    (2, 5),
    (2, 6),
    (2, 8),
    (3, 7),
    (3, 8),
    (4, 7),
    (4, 9),
    (5, 0),
    (5, 2),
    (6, 1),
    (6, 2),
    (7, 1),
    (7, 3),
    (7, 4),
    (8, 2),
    (8, 3),
    (9, 4)
]

In [24]:
print([code[e] for e in resultat])

[0, 5, 2, 8, 3, 7, 4, 9]